### Importing datasets

In [75]:
from pyspark.sql import SQLContext,Row
from pyspark.sql import functions as F

dataDir = "/home/rsk/Documents/Spark"

In [76]:
userData = sc.textFile(dataDir+"/ml-100k/u.user").map(lambda x : x.split("|"))

In [77]:
movieData = sc.textFile(dataDir+"/ml-100k/u.item").map(lambda x : x.split("|"))

In [78]:
ratingData = sc.textFile(dataDir+"/ml-100k/u.data").map(lambda x : x.split("\t"))

In [79]:
#TO-DO : Convert to Spark DataFrame

ratingDataDF = ratingData.map(lambda x : Row(userID = int(x[0]),
                        movieID = int(x[1]),
                        rating=float(x[2]),
                        timestamp = int(x[3])))
ratingDataDF = sqlContext.createDataFrame(ratingDataDF)

In [80]:
userDataDF = userData.map(lambda x : Row(userID=int(x[0]),
                                        age = int(x[1]),
                                        gender = x[2],
                                        occupation = x[3],
                                        zipcode = x[4]))
userDataDF = sqlContext.createDataFrame(userDataDF)

In [81]:
movieDataDF = movieData.map(lambda x : Row(movieID = int(x[0]),
                                            movieTitle = x[1],
                                            releaseDate = x[2],
                                            videoReleaseDate = x[3],
                                            IMDBurl = x[4],
                                            unknown= int(x[5]),
                                            action = int(x[6]),
                                            adventure = int(x[7]),
                                            animation = int(x[8]),
                                            childrens = int(x[9]),
                                            comedy = int(x[10]),
                                             crime = int(x[11]),
                                             documentary = int(x[12]),
                                             drama = int(x[13]),
                                             fantasy = int(x[14]),
                                             filmNoir = int(x[15]),
                                             horror = int(x[16]),
                                             musical = int(x[17]),
                                             mystery = int(x[18]),
                                             romance = int(x[19]),
                                             sciFi = int(x[20]),
                                             thriller = int(x[21]),
                                             war = int(x[22]),
                                             western = int(x[23])))
movieDataDF = sqlContext.createDataFrame(movieDataDF)
                                             

In [84]:
ratingDataDF.show(5)

+-------+------+---------+------+
|movieID|rating|timestamp|userID|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
|     51|   2.0|880606923|   244|
|    346|   1.0|886397596|   166|
+-------+------+---------+------+
only showing top 5 rows



In [86]:
#Taking smaller subsets of original dataset

u = sqlContext.createDataFrame(userDataDF.rdd.zipWithIndex().filter(lambda x : x[1]<15).map(lambda x : x[0]))



In [95]:
userID = u.map(lambda x : x.userID).collect()

r = ratingDataDF[ratingDataDF.userID.isin(userID)][ratingDataDF.columns]

In [97]:
r.count()

2536

In [98]:
userID

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [ ]:
r = ratingsDF

In [20]:
ratings = sqlContext.createDataFrame(ratings)

In [21]:
ratings.show()

+-------+------+---------+------+
|movieID|rating|timestamp|userID|
+-------+------+---------+------+
|    242|   3.0|881250949|   196|
|    302|   3.0|891717742|   186|
|    377|   1.0|878887116|    22|
|     51|   2.0|880606923|   244|
|    346|   1.0|886397596|   166|
|    474|   4.0|884182806|   298|
|    265|   2.0|881171488|   115|
|    465|   5.0|891628467|   253|
|    451|   3.0|886324817|   305|
|     86|   3.0|883603013|     6|
|    257|   2.0|879372434|    62|
|   1014|   5.0|879781125|   286|
|    222|   5.0|876042340|   200|
|     40|   3.0|891035994|   210|
|     29|   3.0|888104457|   224|
|    785|   3.0|879485318|   303|
|    387|   5.0|879270459|   122|
|    274|   2.0|879539794|   194|
|   1042|   4.0|874834944|   291|
|   1184|   2.0|892079237|   234|
+-------+------+---------+------+
only showing top 20 rows



In [31]:
ratings

Column<rating[data]>

In [20]:
schema = sqlContext.createDataFrame(user_data)

In [21]:
schema.show()

+---+---+---+-------------+-----+
| _1| _2| _3|           _4|   _5|
+---+---+---+-------------+-----+
|  1| 24|  M|   technician|85711|
|  2| 53|  F|        other|94043|
|  3| 23|  M|       writer|32067|
|  4| 24|  M|   technician|43537|
|  5| 33|  F|        other|15213|
|  6| 42|  M|    executive|98101|
|  7| 57|  M|administrator|91344|
|  8| 36|  M|administrator|05201|
|  9| 29|  M|      student|01002|
| 10| 53|  M|       lawyer|90703|
| 11| 39|  F|        other|30329|
| 12| 28|  F|        other|06405|
| 13| 47|  M|     educator|29206|
| 14| 45|  M|    scientist|55106|
| 15| 49|  F|     educator|97301|
| 16| 21|  M|entertainment|10309|
| 17| 30|  M|   programmer|06355|
| 18| 35|  F|        other|37212|
| 19| 40|  M|    librarian|02138|
| 20| 42|  F|    homemaker|95660|
+---+---+---+-------------+-----+
only showing top 20 rows



In [23]:
df = schema
df.filter(df['_2']>50).show()

+---+---+---+-------------+-----+
| _1| _2| _3|           _4|   _5|
+---+---+---+-------------+-----+
|  2| 53|  F|        other|94043|
|  7| 57|  M|administrator|91344|
| 10| 53|  M|       lawyer|90703|
| 47| 53|  M|    marketing|07102|
| 65| 51|  F|     educator|48118|
| 85| 51|  M|     educator|20003|
| 90| 60|  M|     educator|78155|
| 91| 55|  M|    marketing|01913|
|106| 61|  M|      retired|55125|
|111| 57|  M|     engineer|90630|
|121| 54|  M|    librarian|99603|
|131| 59|  F|administrator|15237|
|133| 53|  M|     engineer|78602|
|136| 51|  M|        other|97365|
|144| 53|  M|   programmer|20910|
|157| 57|  M|     engineer|70808|
|169| 52|  F|        other|53705|
|170| 53|  F|   healthcare|30067|
|172| 55|  M|    marketing|22207|
|173| 56|  M|        other|22306|
+---+---+---+-------------+-----+
only showing top 20 rows



In [24]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string]>

In [26]:
df.columns=['userID','Age','Gender','Occupation','ZipCode']

AttributeError: can't set attribute